In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/chang/Deepface
!pip install -e .
from deepface import DeepFace

import os
from os import path
import warnings
import time
import pickle
import logging

# 3rd party dependencies
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import tensorflow as tf
from deprecated import deprecated

from deepface.basemodels import (
    VGGFace,
    OpenFace,
    Facenet,
    Facenet512,
    FbDeepFace,
    DeepID,
    DlibWrapper,
    ArcFace,
    SFace,
)
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface.commons import functions, realtime, distance as dst

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf_version = int(tf.__version__.split(".", maxsplit=1)[0])
if tf_version == 2:
    tf.get_logger().setLevel(logging.ERROR)

In [4]:
def get_model(model_name):
    global model_dict

    models = {
        "VGG-Face": VGGFace.loadModel,
        "OpenFace": OpenFace.loadModel,
        "Facenet": Facenet.loadModel,
        "Facenet512": Facenet512.loadModel,
        "DeepFace": FbDeepFace.loadModel,
        "DeepID": DeepID.loadModel,
        "Dlib": DlibWrapper.loadModel,
        "ArcFace": ArcFace.loadModel,
        "SFace": SFace.load_model,
    }

    if not "model_dict" in globals():
        model_dict = {}

    if not model_name in model_dict:
        model = models.get(model_name)
        if model:
            model = model()
            model_dict[model_name] = model
        else:
            raise ValueError(f"Invalid model_name passed - {model_name}")

    return model_dict[model_name]


In [2]:
%cd /content/drive/MyDrive/chang
from preprocess_data import *
train_ds, val_ds = make_dataset("/content/drive/MyDrive/chang/dataset/align_image_v2", include_label=False)
len(train_ds)

/content/drive/MyDrive/chang
image count :  7884
classes num :  1006
['0' '0 (1)' '1' ... '997' '998' '999']
train image count :  7101
val image count :  788
Image shape:  (112, 3)
Label:  479


111

In [ ]:
%cd /content/drive/MyDrive/chang/arcface-tf2/

from modules.models import ArcFaceModel
from modules.losses import SoftmaxLoss
from modules.utils import set_memory_growth, load_yaml, get_ckpt_inf
import modules.dataset as dataset
from modules.models import *
from tensorflow import keras
from tensorflow.keras import layers

model = get_model("ArcFace")

for layer in model.layers[0:-5]:
    layer.trainable=False

inputs = Input([112, 112, 3], name='input_image')
x = model(inputs, training=False)
labels = Input([], name='label')
logist = ArcHead(num_classes=1005, margin=0.5,logist_scale=64)(x, labels)
tmodel = Model((inputs, labels), logist, name='arcface_model')
'''
logist = NormHead(num_classes=1005, w_decay=5e-4)(x)
tmodel = Model(inputs, logist, name='arcface_normhead')
'''
tmodel.compile()
tmodel.summary()

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
tmodel.compile(optimizer=optimizer, loss=SoftmaxLoss(), metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history=tmodel.fit(train_ds, epochs=150, callbacks=[callback], validation_data=val_ds, steps_per_epoch=111)

In [ ]:
'''
tempinputs = tmodel.input[0]
temp_model = tmodel.layers[1](tmodel.layers[0].output)
temp_model = Model(ttinputs, temp_model)
temp_model.compile()
temp_model.summary()
'''
tempinputs = Input([112, 112, 3])
ttmodel = tmodel.layers[1](tempinputs)
ttmodel = Model(tempinputs, ttmodel)
ttmodel.compile()
ttmodel.summary()